# Amazon SageMaker - Bring Your Own Model 
## TensorFlow + Keras 編

ここでは TensorFlow と Keras を使ったサンプルコードを題材に、Amazon SageMaker 移行の方法を順を追って説明します。SageMaker Python SDK で TensorFlow を使うための説明は [SDK のドキュメント](https://sagemaker.readthedocs.io/en/stable/using_tf.html) にも多くの情報があります。

注: 
ここで説明するのは Script モード という記法 (現時点では標準の書き方) で、FILE モード (入力データを Amazon S3 から学習時にファイルとしてコピーする方法) です。データサイズが大きくなった場合は、FILE Mode ではなく PIPE Mode をお使い頂いた方がスループットが向上します。
また、ここでは以降手順の紹介のためトレーニングスクリプトは最小限の書き換えとしています。

## 1. トレーニングスクリプトの書き換え
まず [サンプルのソースコード](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py) を以下のコマンドでダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/keras-team/keras/master/examples/mnist_mlp.py

このブログにあるように書き換えます。(あとでこのノートブックに説明を書き足します。)

https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-keras/

## 2. Notebook 上でのデータ準備

トレーニングスクリプトの書き換えは終了しました。トレーニングを始める前に、予め Amazon S3 にデータを準備しておく必要があります。この Notebook を使ってその作業をします。

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import mnist

import sagemaker

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

In [ ]:
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()

input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='data/handson-byom-tensorflow-keras')

print('Input data is uploaded to: {}'.format(input_data))

## トレーニングジョブの発行

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()
estimator = TensorFlow(entry_point = "./mnist_mlp.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type="ml.c5.xlarge",
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 4})

estimator.fit(input_data)

### 学習済みモデルの確認

In [ ]:
!aws s3 cp $estimator.model_data ./
!tar zxvf model.tar.gz

TensorFlow `SavedModel` 形式で保存されたモデルを読み込みます。

In [ ]:
with tf.Session() as sess:
    tf.saved_model.load(sess, [tf.saved_model.tag_constants.SERVING], "model/1/")
    
    i = sess.graph.get_tensor_by_name('dense_1_input:0')
    o = sess.graph.get_tensor_by_name('dense_3/Softmax:0')
    
    pred = sess.run(o, feed_dict={i:x_test[:10].reshape(-1, 784)})
    print('pred:', np.argmax(pred, axis=1))
    
print('true:', y_test[:10])

## 推論エンドポイントのデプロイ

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_test.shape[0] - 1), num_samples)
images, labels = x_test[indices]/255, y_test[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

In [ ]:
predictor.delete_endpoint()